In [1]:
import streamlit as st
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain. prompts import PromptTemplate
from langchain. chains import LLMChain
from langchain_community.document_loaders import CSVLoader

# 

In [2]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv


C:\Users\otimi\anaconda3\envs\python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv(dotenv_path='.env')

# https://platform.openai.com/api-keys
openai_api_key = os.getenv("OPENAI_API_KEY")

# https://console.groq.com/keys
groq_api_key = os.getenv("GROQ_API_KEY")

# https://console.anthropic.com/dashboard
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

# https://aistudio.google.com/app/apikey
gemini_api_key = os.getenv("GEMINI_API_KEY")


In [ ]:
# Não execute
loader = CSVLoader(file_path="lei.txt", encoding='utf-8')
documents = loader.load()
documents

In [ ]:
# Não execute
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
db = FAISS.from_documents(documents, embeddings)

In [ ]:
# Não execute
question = "Onde é o domicílio da pessoa natural ?"
similar_response = db.similarity_search(question,k=3)
best_practice = [doc.page_content for doc in similar_response]
best_practice

In [4]:
template = """
Você é um assistente virtual de um escritório de advocacia.
Sua função será responder e-mails que recebemos de clientes respondendo perguntas sobre a lei do Novo Código Civil
O novo Código Civil é estabelecido pela LEI Nº 10.406, de 10 de janeiro de 2002.

Siga todas as regras abaixo de acordo com contexto {context} lavando em conta o histórico {chat_history}:
1. Você deve buscar se comportar de maneira cordial e solícita.
2. Suas respostas devem ser bem similares ou até identicas aos termos da lei.
3. Alguns dos e-mails podem conter links e informações irrelevantes. Preste atenção apenas no conteúdo útil da mensagem.
Aqui está uma mensagem da dúvida recebida de um cliente.
{question}

Escreva a melhor resposta que eu deveria enviar para solucionar a dúvida apresentada pela pergunta deste cliente.
Encerre com a seguinte assinatura:

Equipe Fale Conosco 
email de contato: faleconosco@email.com.br
"""

In [5]:
from langchain.prompts.prompt import PromptTemplate
prompt = PromptTemplate(input_variables=['question','context','chat_history'],template=template)

In [ ]:
# Não execute
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)
chain = prompt | llm
response = chain.invoke({"message":question,"best_practice":best_practice})
print(response.content)

In [6]:
from langchain.document_loaders import PyPDFLoader ,TextLoader

def load_doc(document_path):
    loader = PyPDFLoader(document_path)
    documents = loader.load()
    # Split document in chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
    docs = text_splitter.split_documents(documents=documents)
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    # Create vectors
    vectorstore = FAISS.from_documents(docs, embeddings)
    # Persist the vectors locally on disk
    vectorstore.save_local("faiss_index_datamodel")

    # Load from local storage
    persisted_vectorstore = FAISS.load_local("faiss_index_datamodel", embeddings, allow_dangerous_deserialization=True)
    return persisted_vectorstore

In [8]:
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
#from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)
# chain = prompt | llm
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=load_doc("./dados/lei.pdf").as_retriever(),
    combine_docs_chain_kwargs={'prompt': prompt},
    chain_type="stuff",
)
question = "Onde é o domicílio da pessoa natural ?"
chat_history=[]
response = chain.invoke({"question":question,"chat_history":chat_history})
print(response['answer'])


C:\Users\otimi\anaconda3\envs\python\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Prezado(a) Senhor(a),

O domicílio da pessoa natural, segundo o Novo Código Civil, é o lugar onde a pessoa estabelece a sua residência com ânimo definitivo. De acordo com o artigo 70 do referido código, "a lei presume como domicílio do empresário o lugar onde ele exerce sua atividade, e, na falta de estabelecimento, o domicílio onde reside".

Portanto, o domicílio é o local que a pessoa considera sua residência habitual, sendo importante ressaltar que a intenção de permanecer no local é fundamental para a caracterização do domicílio.

Caso tenha mais dúvidas ou precise de esclarecimentos adicionais, estamos à disposição para ajudar.

Atenciosamente,

Equipe Fale Conosco  
email de contato: faleconosco@email.com.br
